In [1]:
import torch
import numpy as np
# tensor1 = torch.tensor([1, 3, 5, 7, 9])
# tensor2 = torch.tensor([1, 2, 3, 6, 5])

# result = np.intersect1d(tensor1, tensor2)
# result

In [11]:
from PIL import Image
def find_overlap_cls(obj1: torch.Tensor, obj2: torch.Tensor):
    """
    find overlapped class between two masks
    
    param
     - obj1,2: GT class segmentation map
     
    return: overlapped class mask map
    """

    cls1 = torch.unique(obj1)
    cls2 = torch.unique(obj2)

    overlapped_cls = torch.from_numpy(np.intersect1d(cls1, cls2))
    
    #ovl cls 만 열고 나머지는 -1으로 만들어
    obj_mask1 = torch.where(torch.isin(obj1, overlapped_cls), obj1, -1.)
    obj_mask2 = torch.where(torch.isin(obj2, overlapped_cls), obj2, -1.)


    return obj_mask1, obj_mask2, overlapped_cls

obj1=np.array(Image.open("/home/gaussian-grouping/data/lerf_mask/teatime/object_mask/frame_00001.png").convert("L"))
obj2= np.array(Image.open("/home/gaussian-grouping/data/lerf_mask/teatime/object_mask/frame_00100.png").convert("L"))


obj1=torch.from_numpy(obj1)
obj2=torch.from_numpy(obj2)

m1,m2, ov=find_overlap_cls(obj1, obj2)


In [12]:
torch.unique(m2)

tensor([ -1.,   0.,  25.,  48.,  52.,  59.,  62.,  63.,  73.,  75.,  81.,  97.,
         99., 103., 110., 118.])

In [27]:
def contrastive_loss(features, instance_labels, temperature):
    """
    https://github.com/yashbhalgat/Contrastive-Lift/blob/main/model/loss/loss.py
    compute the language feature's contrastive loss 
    param :
        - f_map : rendered semantic featrue of language 
        - obj_map : target's object instance map

    return :
        computed loss value
    """

    bsize = features.size(0)
    masks = instance_labels.view(-1, 1).repeat(1, bsize).eq_(instance_labels.clone())
    masks = masks.fill_diagonal_(0, wrap=False)

    # compute similarity matrix based on Euclidean distance
    distance_sq = torch.pow(features.unsqueeze(1) - features.unsqueeze(0), 2).sum(dim=-1)
    # temperature = 1 for positive pairs and temperature for negative pairs
    temperature = torch.ones_like(distance_sq) * temperature
    temperature = torch.where(masks==1, temperature, torch.ones_like(temperature))

    similarity_kernel = torch.exp(-distance_sq/temperature)
    logits = torch.exp(similarity_kernel)

    p = torch.mul(logits, masks).sum(dim=-1)
    Z = logits.sum(dim=-1)

    prob = torch.div(p, Z)
    prob_masked = torch.masked_select(prob, prob.ne(0))
    loss = -prob_masked.log().sum()/bsize

    return loss

In [16]:
import random
def _sample_indices(num_samples, size):
    indices = random.sample(range(size), min(num_samples, size))
    return torch.tensor(indices)

def _rbf_kernel(x1, x2, gamma=1.0):
    diff = x1.unsqueeze(1) - x2.unsqueeze(0)
    squared_diff = torch.sum(diff**2, dim=-1)
    return torch.exp(-gamma * squared_diff)

def contrastive_loss(features, objects, margin=1.0, gamma=1.0, num_samples=1000):
    loss = 0.0
    for class_id in torch.unique(objects):
        class_objects = (objects == class_id).float()
        num_objects = torch.sum(class_objects)
        
        class_features = features * class_objects.unsqueeze(-1)
        
        if num_objects <= 1:
            continue
        
        #sample_indices = _sample_indices(num_samples, num_objects)
        sampled_indices = torch.randint(0, features.shape[0] * features.shape[1], (num_samples,))
        class_features = class_features.view(-1, class_features.shape[-1]) # (H*W , 3)
        sampled_class_features = class_features[sampled_indices]
        print(sampled_class_features, sampled_indices)
        if num_samples <= 100:
            ValueError("num_sample is too low")
        
        positive_similarities = _rbf_kernel(sampled_class_features, sampled_class_features, gamma)
        positive_similarities = positive_similarities.triu(diagonal=1)
        num_positive_pairs = torch.sum(positive_similarities > 0)
        
        negative_similarities = _rbf_kernel(sampled_class_features, features, gamma)
        negative_similarities = negative_similarities * (1 - class_objects.unsqueeze(-1))
        
        loss += torch.sum(torch.relu(margin - positive_similarities)) / num_positive_pairs
        loss += torch.sum(torch.relu(negative_similarities - margin)) / (num_samples * (features.size(0) - num_samples))
        
    return loss

In [97]:
loss = 0.0
objects = m1
features = lang_f.clone()
num_samples =4096
gamma = 0.01
for class_id in torch.unique(objects): # obj id 0인경우 > class_objects에서 마스킹 됨
    class_objects = (objects == class_id).float() # 해당 클래스에 대한 픽셀들
    num_obj_pixel = torch.sum(class_objects)
    
    class_features = features * class_objects.unsqueeze(-1)
    class_features = torch.where(class_features>=0, class_features, 0) 
    if num_obj_pixel <= 1:
        continue
    
    #sample_indices = _sample_indices(num_samples, num_objects)
    sampled_indices = torch.randint(0, features.shape[0] * features.shape[1], (num_samples,)) #u개의 픽셀을 "전체에서" 뽑음 
    
    class_features = class_features.view(-1, class_features.shape[-1]) # (H*W , 3)
    sampled_cls_indices = sampled_indices[class_objects.view(-1,1)[sampled_indices].squeeze(-1) == 1.] #샘플링한 픽셀 중에서 클래스에 해당하는 픽셀들 
    sampled_class_features = class_features[sampled_cls_indices] # pos pair 

    
    positive_similarities = _rbf_kernel(sampled_class_features, sampled_class_features, gamma) # 샘플링을 같은 클래스에서 해야함
    
    positive_similarities = positive_similarities.triu(diagonal=1)
    num_positive_pairs = torch.sum(positive_similarities > 0)
    
    features_flatten = features.view(-1, features.shape[-1])
    sampled_features = features_flatten[sampled_indices]
    
    negative_similarities = _rbf_kernel(sampled_features, sampled_features, gamma)
    negative_similarities = negative_similarities.triu(diagonal=1)
    num_total_pairs = torch.sum(negative_similarities > 0)

    #negative_similarities = negative_similarities * (1 - class_objects.unsqueeze(-1))
    
    # positive_similarities= torch.exp(positive_similarities).sum()
    # negative_similarities= torch.exp(negative_similarities).sum()

    # logits= -torch.log(positive_similarities/negative_similarities) / num_total_pairs
    
    # # Positive pairs loss
    # if num_positive_pairs > 0:
    #     positive_loss = -torch.log(positive_similarities.sum() / num_positive_pairs)
    #     loss += positive_loss
    
    # # Negative pairs loss
    # if (1 - class_objects).sum() > 0:
    #     negative_loss = -torch.log(negative_similarities.sum() / (1 - class_objects).sum())
    #     loss += negative_loss
    
#loss /= len(torch.unique(objects))

In [99]:
loss = 0.0
objects = m1
features = lang_f.clone()
num_samples = 4096
gamma = 0.01

for class_id in torch.unique(objects):
    class_objects = (objects == class_id).float()
    num_obj_pixel = torch.sum(class_objects)
    
    if num_obj_pixel <= 1:
        continue
    
    class_features = features * class_objects.unsqueeze(-1)
    class_features = torch.where(class_features >= 0, class_features, 0) 

    # Sample indices from all pixels
    sampled_indices = torch.randint(0, features.shape[0] * features.shape[1], (num_samples,))
    
    class_features = class_features.view(-1, class_features.shape[-1])
    # Filter out sampled indices that belong to the current class
    sampled_cls_indices = sampled_indices[class_objects.view(-1,1)[sampled_indices].squeeze(-1) == 1.] 
    sampled_class_features = class_features[sampled_cls_indices]

    # Compute positive similarities
    positive_similarities = _rbf_kernel(sampled_class_features, sampled_class_features, gamma) 
    num_positive_pairs = torch.sum(positive_similarities > 0)
    
    # Compute negative similarities between features of different classes
    negative_class_ids = torch.unique(objects[objects != class_id])
    sampled_neg_indices = torch.randint(0, features.shape[0] * features.shape[1], (num_samples,))
    sampled_neg_features = features.view(-1, features.shape[-1])[sampled_neg_indices]
    sampled_neg_class_features = torch.stack([class_features[objects == neg_id] for neg_id in negative_class_ids])
    negative_similarities = torch.cat([_rbf_kernel(sampled_class_features, sampled_neg_features, gamma) for sampled_class_features in sampled_neg_class_features])
    num_negative_pairs = torch.sum(negative_similarities > 0)
    
    # Compute logits for contrastive loss
    logits = -torch.log(num_positive_pairs / num_negative_pairs)
    
    loss += logits

# Normalize the loss by the number of classes
loss /= len(torch.unique(objects))

IndexError: The shape of the mask [730, 988] at index 0 does not match the shape of the indexed tensor [721240, 3] at index 0

In [20]:
torch.unique(class_objects)

tensor([0., 1.])

In [79]:
import torch
lang_f = torch.rand(730,988,3)
m1,m2, ovl_cls=find_overlap_cls(obj1, obj2)
# m1, m2는 마스크 , ovl cls는 인덱스 리스트
#m1의 각각의 인스탄스 영역들에 대해서, -1 값인 것은 제외시키고 나머지 클래스들에 대해서 분리(마스크) 
# 각 클래스의 인스탄스 영역에 대해서 

#contrastive_loss(lang_f, m1)